# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
pip install hyperopt lightgbm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK


### 데이터 읽어오기


In [2]:
ROOT_DIR = "./data/"
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

X Judge Value_Dam -> X Collect Result가 NaN이면 NG, 숫자면 NaN으로 결측치 보간

In [3]:
# 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam' 컬럼의 결측치를 조건에 따라 채우기   -> 이거 안돼있는데??
train['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'] = train['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'].combine_first(
    train['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].apply(lambda x: 'NG' if pd.isna(x) else np.nan)
)

test['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'] = test['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'].combine_first(
    test['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].apply(lambda x: 'NG' if pd.isna(x) else np.nan)
)

fill_nan = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 
            'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 
           'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']
for col in fill_nan:
  train[col] = train[col].replace('OK', np.nan)
  test[col] = test[col].replace('OK', np.nan)

전부 NA이거나 unique값이 1이면서 NA가 없는 컬럼들 drop. OK만 들어가 있는 변수들은 Judge value Dam과 모두 겹치므로 drop.

남는 컬럼 : 147개

In [4]:
drop_col = []

for col in train.columns:
  nullcount = train[col].isnull().sum()
  nunique = train[col].nunique()
  if nullcount == len(train):
    drop_col.append(col)
  if (nunique==1) & (nullcount == 0):
    drop_col.append(col)

# 겹치는 OK변수들 제거
drop_col.extend(['HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1',
                 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
                 'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
                 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave']
                 )
# drop column
print('전체 컬럼 수 :', len(train.columns))
print('drop할 컬럼 수 :', len(drop_col))
print('사용 컬럼 수 :', len(train.columns) - len(drop_col))

train = train.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)


전체 컬럼 수 : 464
drop할 컬럼 수 : 317
사용 컬럼 수 : 147


### (X,Y,Z) 좌표 변수 처리

head 공정 X, Y, Z 축 별로 연결 (dam, fill1, fill2)

In [5]:
head_coordinate_col = [[
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
 'HEAD Standby Position X Collect Result_Dam',
 'HEAD Standby Position Y Collect Result_Dam',
 'HEAD Standby Position Z Collect Result_Dam',
 'Head Clean Position X Collect Result_Dam',
 'Head Clean Position Y Collect Result_Dam',
 'Head Clean Position Z Collect Result_Dam',
 'Head Purge Position X Collect Result_Dam',
 'Head Purge Position Y Collect Result_Dam',
 'Head Purge Position Z Collect Result_Dam',
 'Head Zero Position X Collect Result_Dam',
 'Head Zero Position Y Collect Result_Dam',
 'Head Zero Position Z Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
 'HEAD Standby Position X Collect Result_Fill1',
 'HEAD Standby Position Y Collect Result_Fill1',
 'HEAD Standby Position Z Collect Result_Fill1',
 'Head Clean Position X Collect Result_Fill1',
 'Head Clean Position Y Collect Result_Fill1',
 'Head Clean Position Z Collect Result_Fill1',
 'Head Purge Position X Collect Result_Fill1',
 'Head Purge Position Y Collect Result_Fill1',
 'Head Purge Position Z Collect Result_Fill1',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2',
 'HEAD Standby Position X Collect Result_Fill2',
 'HEAD Standby Position Y Collect Result_Fill2',
 'HEAD Standby Position Z Collect Result_Fill2',
 'Head Clean Position X Collect Result_Fill2',
 'Head Clean Position Y Collect Result_Fill2',
 'Head Clean Position Z Collect Result_Fill2',
 'Head Purge Position X Collect Result_Fill2',
 'Head Purge Position Y Collect Result_Fill2',
 'Head Purge Position Z Collect Result_Fill2']]

In [6]:
pos_col = head_coordinate_col[0].copy()

grouped_columns = {}
grouped_columns['Dam'] = {}
grouped_columns['Fill1'] = {}
grouped_columns['Fill2'] = {}

# 그룹핑
for column in pos_col:
  if 'judge' not in column.lower():
    parts = column.split('_')
    if len(parts) == 2:
        coord_info, process_type = parts
        axis = 'X' if ' X ' in coord_info else 'Y' if ' Y ' in coord_info else 'Z' if ' Z ' in coord_info else 'Θ' if ' Θ ' in coord_info else 'Unknown'
        # axis가 존재하지 않으면 새로 생성
        if axis not in grouped_columns[process_type]:
            grouped_columns[process_type][axis] = []
        grouped_columns[process_type][axis].append(column)

# 딕셔너리의 각 프로세스와 축에 대해 새로운 컬럼 생성
for process_type, axes in grouped_columns.items():
    for axis, columns in axes.items():
        # 각 축에 대한 컬럼을 tuple로 묶어 새로운 컬럼을 생성
        new_column_name = f'{process_type} {axis}'  # ex: 'Dam X', 'Fill1 X'
        train[new_column_name] = list(zip(*[train[col] for col in columns]))
        train[new_column_name] = train[new_column_name].astype(str)
        test[new_column_name] = list(zip(*[test[col] for col in columns]))
        test[new_column_name] = test[new_column_name].astype(str)

head 공정 중 clean - purge X,Y,Z 축 별로 연결 (Dam, Fill1 만)

In [7]:
cleanpurge_col_set = [['Head Clean Position X Collect Result_Fill1',
'Head Purge Position X Collect Result_Fill1'],
                 ['Head Clean Position Y Collect Result_Fill1',
'Head Purge Position Y Collect Result_Fill1'],
                 ['Head Clean Position Z Collect Result_Fill1',
'Head Purge Position Z Collect Result_Fill1'],
                 
                 ['Head Clean Position X Collect Result_Dam',
'Head Purge Position X Collect Result_Dam'],
                 ['Head Clean Position Y Collect Result_Dam',
'Head Purge Position Y Collect Result_Dam'],
                 ['Head Clean Position Z Collect Result_Dam',
'Head Purge Position Z Collect Result_Dam']]

new_col = []
for col_set in cleanpurge_col_set:
  colname = col_set[0].replace('Clean', 'Clean&Purge')
  new_col.append(colname)
  train[colname] = list(train[col_set].itertuples(index=False, name=None))
  train[colname] = train[colname].astype(str)
  test[colname] = list(test[col_set].itertuples(index=False, name=None))
  test[colname] = test[colname].astype(str)

# drop_col = list(set(drop_col + sum(stage_col_set, [])))
# len(drop_col)

cure 공정 연결가능한 축끼리 연결 (Dam은 X,Θ / Fill2는 X,Z)

없는 좌표는 nunique가 하나라 위에서 drop 당한거

In [8]:
cure_col_set = [ ['CURE START POSITION X Collect Result_Dam',
                  'CURE END POSITION X Collect Result_Dam'],
                 ['CURE START POSITION Θ Collect Result_Dam',
                  'CURE END POSITION Θ Collect Result_Dam'],
                 [ 'CURE START POSITION X Collect Result_Fill2',
                   'CURE END POSITION X Collect Result_Fill2'],
                 [ 'CURE START POSITION Z Collect Result_Fill2',
                   'CURE END POSITION Z Collect Result_Fill2']]



new_col = []
for col_set in cure_col_set:
  colname = col_set[0].replace('START', '')
  new_col.append(colname)
  train[colname] = list(train[col_set].itertuples(index=False, name=None))
  train[colname] = train[colname].astype(str)
  test[colname] = list(test[col_set].itertuples(index=False, name=None))
  test[colname] = test[colname].astype(str)

# drop_col = list(set(drop_col + sum(stage_col_set, [])))
# len(drop_col)

head 공정 stage 123 X,Y,Z 축기리 연결 

In [9]:
stage_col_set = [['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'],
                 ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'],
                 ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'],
                 ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'],
                 ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'],
                 ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'],
                 ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'],
                 ['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'],
                 ['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2',
                  'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2']]
new_col = []
for col_set in stage_col_set:
  colname = col_set[0].replace('1)', ')')
  new_col.append(colname)
  train[colname] = list(train[col_set].itertuples(index=False, name=None))
  train[colname] = train[colname].astype(str)
  test[colname] = list(test[col_set].itertuples(index=False, name=None))
  test[colname] = test[colname].astype(str)

# drop_col = list(set(drop_col + sum(stage_col_set, [])))
# len(drop_col)

모든 head 공정 좌표에 대하여 (X,Y,Z) 좌표 생성 

In [10]:
import re
coordinate_groups = {}

head_dam_XYZ_col = []
head_fill1_XYZ_col = []
head_fill2_XYZ_col = []

cure_XYZ_col = []

# filtered_columns = [col for col in column_names if 'CURE' not in col]




for col in pos_col:
    # 정규 표현식을 사용하여 X, Y, Z 부분을 찾고, 공통 이름을 추출
    match = re.match(r'(.*?)(\bX\b|\bY\b|\bZ\b)(.*)', col)
    if match:
        base_name = match.group(1).strip() + ' ' + match.group(3).strip()
        if base_name not in coordinate_groups:
            coordinate_groups[base_name] = {}
        coordinate_groups[base_name][match.group(2)] = col

# 각 그룹에 대해 가능한 X, Y, Z 좌표를 묶어서 새로운 컬럼 추가
for base_name, coords in coordinate_groups.items():
    # 기존 컬럼 중 존재하는 X, Y, Z 값을 가져와서 묶기
    grouped_columns_train = []
    grouped_columns_test = []
    for coord in ['X', 'Y', 'Z']:
        if coord in coords:
            grouped_columns_train.append(train[coords[coord]])
            grouped_columns_test.append(test[coords[coord]])
        else:
            grouped_columns_train.append([None] * len(train))  # 해당 컬럼이 없으면 None으로 채움
            grouped_columns_test.append([None] * len(test))
    # 새로운 (X, Y, Z) 튜플 컬럼 생성
    train[f'{base_name} XYZ'] = [str(tup) for tup in list(zip(*grouped_columns_train))] 
    test[f'{base_name} XYZ'] = [str(tup) for tup in list(zip(*grouped_columns_test))]
    if 'head' in base_name.lower():
        if 'dam' in base_name.lower():
            head_dam_XYZ_col.append(f'{base_name} XYZ')
        if 'fill1' in base_name.lower():
            head_fill1_XYZ_col.append(f'{base_name} XYZ')
        if 'fill2' in base_name.lower():
            head_fill2_XYZ_col.append(f'{base_name} XYZ')
    
            
    # else:
    #     cure_XYZ_col.append(f'{base_name} XYZ')
    
    # 기존의 X, Y, Z 컬럼 제거
    train.drop(columns=list(coords.values()), inplace=True)
    test.drop(columns=list(coords.values()), inplace=True)

모든 cure 공정 좌표에 대하여 X_Z_Θ 좌표 생성 

In [11]:
cure_coordinate_col = [col for col in train.columns if 'CURE' in col]
cure_coordinate_col = [col for col in cure_coordinate_col if 'SPEED' not in col]

cure_coordinate_col = cure_coordinate_col[:-4]

train[cure_coordinate_col] = train[cure_coordinate_col].astype(str)
test[cure_coordinate_col] = test[cure_coordinate_col].astype(str)

cure_coordinate_col

['CURE END POSITION X Collect Result_Dam',
 'CURE END POSITION Z Collect Result_Dam',
 'CURE END POSITION Θ Collect Result_Dam',
 'CURE START POSITION X Collect Result_Dam',
 'CURE START POSITION Θ Collect Result_Dam',
 'CURE END POSITION X Collect Result_Fill2',
 'CURE END POSITION Z Collect Result_Fill2',
 'CURE STANDBY POSITION Z Collect Result_Fill2',
 'CURE START POSITION X Collect Result_Fill2',
 'CURE START POSITION Z Collect Result_Fill2']

In [12]:
train['CURE START POSITION Collect Result_Dam XΘ'] = train['CURE START POSITION X Collect Result_Dam'] + '_' + train['CURE START POSITION Θ Collect Result_Dam']
train['CURE END POSITION Collect Result_Dam XZΘ'] = train['CURE END POSITION X Collect Result_Dam'] + '_' + train['CURE END POSITION Z Collect Result_Dam'] +  '_' + train['CURE END POSITION Θ Collect Result_Dam']

train['CURE START POSITION Collect Result_Fill2 XZ'] = train['CURE START POSITION X Collect Result_Fill2'] + '_' + train['CURE START POSITION Z Collect Result_Fill2']
train['CURE END POSITION Collect Result_Fill2 XZ'] = train['CURE END POSITION X Collect Result_Fill2'] + '_' + train['CURE END POSITION Z Collect Result_Fill2'] 

test['CURE START POSITION Collect Result_Dam XΘ'] = test['CURE START POSITION X Collect Result_Dam'] + '_' + test['CURE START POSITION Θ Collect Result_Dam']
test['CURE END POSITION Collect Result_Dam XZΘ'] = test['CURE END POSITION X Collect Result_Dam'] + '_' + test['CURE END POSITION Z Collect Result_Dam'] +  '_' + test['CURE END POSITION Θ Collect Result_Dam']

test['CURE START POSITION Collect Result_Fill2 XZ'] = test['CURE START POSITION X Collect Result_Fill2'] + '_' + test['CURE START POSITION Z Collect Result_Fill2']
test['CURE END POSITION Collect Result_Fill2 XZ'] = test['CURE END POSITION X Collect Result_Fill2'] + '_' + test['CURE END POSITION Z Collect Result_Fill2'] 



train.drop(columns=cure_coordinate_col, inplace=True)

test.drop(columns=cure_coordinate_col, inplace=True)



In [13]:
head_dam_XYZ_col

['HEAD NORMAL COORDINATE AXIS(Stage1) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage1) Judge Value_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage2) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage3) Collect Result_Dam XYZ',
 'HEAD Standby Position Collect Result_Dam XYZ',
 'Head Clean Position Collect Result_Dam XYZ',
 'Head Purge Position Collect Result_Dam XYZ',
 'Head Zero Position Collect Result_Dam XYZ']

In [14]:
del head_dam_XYZ_col[1]  #Judge 제외
del head_dam_XYZ_col[-1] # Zero 제외
 

head_XYZ_col = head_dam_XYZ_col + head_fill1_XYZ_col + head_fill2_XYZ_col 

head_dam_XYZ_col = [head_dam_XYZ_col[3]] + head_dam_XYZ_col[:3] + head_dam_XYZ_col[4:]
head_fill1_XYZ_col = [head_fill1_XYZ_col[3]] + head_fill1_XYZ_col[:3] + head_fill1_XYZ_col[4:]
head_fill2_XYZ_col = [head_fill2_XYZ_col[3]] + head_fill2_XYZ_col[:3] + head_fill2_XYZ_col[4:]

In [15]:
cure_XYZ_col = train.columns[-4:].tolist()

In [16]:
head_dam_XYZ_col

['HEAD Standby Position Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage1) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage2) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage3) Collect Result_Dam XYZ',
 'Head Clean Position Collect Result_Dam XYZ',
 'Head Purge Position Collect Result_Dam XYZ']

In [17]:
head_dam_XYZ_col

['HEAD Standby Position Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage1) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage2) Collect Result_Dam XYZ',
 'HEAD NORMAL COORDINATE AXIS(Stage3) Collect Result_Dam XYZ',
 'Head Clean Position Collect Result_Dam XYZ',
 'Head Purge Position Collect Result_Dam XYZ']

head interaction: head 공정 Stage1,2,3,Standby,Clean,Purge (X,Y,Z) 전체연결 (Dam,Fill1,Fill2 분리 X)

stage123 : head 공정 중 stage123만 (X,Y,Z) 연결 (Dam,Fill1,Fill2 분리 O)

clean_purge: head 공정 중 clean_purge만 (X,Y,Z) 연결 (Dam,Fill1,Fill2 분리 O)

In [18]:
from sklearn.preprocessing import LabelEncoder

# 1. 라벨 인코딩 적용
label_encoders = {}
for col in head_XYZ_col:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    for label in list(test[col].unique()): #train에 없는 test값 처리
        if label not in list(le.classes_):
            le.classes_ = np.append(le.classes_,label)
    test[col] = le.transform(test[col])
    label_encoders[col] = le
    train[col] = train[col].astype(str) # 범주 변수 처리
    test[col] = test[col].astype(str) # 범주 변수 처리
    
# 2. 전체 상호작용항 생성 및 라벨 인코딩
interaction_col_name = 'head_interaction'  # 향후에 head_interaction dam/fill1/fill2로 분리해 볼 필요 있음

train[interaction_col_name] = train[head_XYZ_col].astype(str).agg('_'.join, axis=1)
test[interaction_col_name] = test[head_XYZ_col].astype(str).agg('_'.join, axis=1)

# 2. 순차적 상호작용

train['stage123_dam'] = train[head_dam_XYZ_col].iloc[:,1] + '_' + train[head_dam_XYZ_col].iloc[:,2] + '_' + train[head_dam_XYZ_col].iloc[:,3]
train['clean_purge_dam'] = train[head_dam_XYZ_col].iloc[:,4] + '_' + train[head_dam_XYZ_col].iloc[:,4] 

train['stage123_fill1'] = train[head_fill1_XYZ_col].iloc[:,1] + '_' + train[head_fill1_XYZ_col].iloc[:,2] + '_' + train[head_fill1_XYZ_col].iloc[:,3]
train['clean_purge_fill1'] = train[head_fill1_XYZ_col].iloc[:,4] + '_' + train[head_fill1_XYZ_col].iloc[:,5] 

train['stage123_fill2'] = train[head_fill2_XYZ_col].iloc[:,1] + '_' + train[head_fill2_XYZ_col].iloc[:,2] + '_' + train[head_fill2_XYZ_col].iloc[:,3]
train['clean_purge_fill2'] = train[head_fill2_XYZ_col].iloc[:,4] + '_' + train[head_fill2_XYZ_col].iloc[:,5] 

####################################################################################################################3

test['stage123_dam'] = test[head_dam_XYZ_col].iloc[:,1] + '_' + test[head_dam_XYZ_col].iloc[:,2] + '_' + test[head_dam_XYZ_col].iloc[:,3]
test['clean_purge_dam'] = test[head_dam_XYZ_col].iloc[:,4] + '_' + test[head_dam_XYZ_col].iloc[:,4] 
test['stage123_fill1'] = test[head_fill1_XYZ_col].iloc[:,1] + '_' + test[head_fill1_XYZ_col].iloc[:,2] + '_' + test[head_fill1_XYZ_col].iloc[:,3]
test['clean_purge_fill1'] = test[head_fill1_XYZ_col].iloc[:,4] + '_' + test[head_fill1_XYZ_col].iloc[:,5] 

test['stage123_fill2'] = test[head_fill2_XYZ_col].iloc[:,1] + '_' + test[head_fill2_XYZ_col].iloc[:,2] + '_' + test[head_fill2_XYZ_col].iloc[:,3]
test['clean_purge_fill2'] = test[head_fill2_XYZ_col].iloc[:,4] + '_' + test[head_fill2_XYZ_col].iloc[:,5] 


# 4. 상호작용항에 라벨 인코딩 적용
ordered_interaction_col_name = train.columns[train.columns.str.contains('dam|fill1|fill2')]

for i in ordered_interaction_col_name:

    interaction_le = LabelEncoder()
    train[i] = interaction_le.fit_transform(train[i])
    for label in list(test[i].unique()): #train에 없는 test값 처리
        if label not in list(interaction_le.classes_):
            interaction_le.classes_ = np.append(interaction_le.classes_,label)
    test[i] = interaction_le.transform(test[i])
    train[i] = train[i].astype("object") # 범주 변수 처리
    test[i] = test[i].astype("object") # 범주 변수 처리


cure interaction: cure 공정 start,end X_Z_Θ 전체연결 (Dam,Fill2 분리 X)

start_end : cure 공정 중 start,end 만 X_Z_Θ 연결 (Dam,Fill2 분리 O)

In [19]:

# 1. 라벨 인코딩 적용
label_encoders = {}
for col in cure_XYZ_col:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    for label in list(test[col].unique()): #train에 없는 test값 처리
        if label not in list(le.classes_):
            le.classes_ = np.append(le.classes_,label)
    test[col] = le.transform(test[col])
    label_encoders[col] = le
    train[col] = train[col].astype(str) # 범주 변수 처리
    test[col] = test[col].astype(str) # 범주 변수 처리

# 2. 상호작용항 생성 및 라벨 인코딩
interaction_col_name = 'cure_interaction'
train[interaction_col_name] = train[cure_XYZ_col].astype(str).agg('_'.join, axis=1) 
test[interaction_col_name] = test[cure_XYZ_col].astype(str).agg('_'.join, axis=1)


# 순차적 상호작용항

train['start_end_dam'] = train[cure_XYZ_col].iloc[:,0] + '_' + train[cure_XYZ_col].iloc[:,1]
# train['standby_start_fill2'] = train[cure_XYZ_col].iloc[:,3] + '_' + train[cure_XYZ_col].iloc[:,4]
train['start_end_fill2'] = train[cure_XYZ_col].iloc[:,2] + '_' + train[cure_XYZ_col].iloc[:,3]

test['start_end_dam'] = test[cure_XYZ_col].iloc[:,0] + '_' + test[cure_XYZ_col].iloc[:,1]
# test['standby_start_fill2'] = test[cure_XYZ_col].iloc[:,3] + '_' + test[cure_XYZ_col].iloc[:,4]
test['start_end_fill2'] = test[cure_XYZ_col].iloc[:,2] + '_' + test[cure_XYZ_col].iloc[:,3]


# 3. 조합의 count 개수가 10 이하인 경우 하나의 라벨로 묶기
#interaction_counts = train[interaction_col_name].value_counts()

# 빈도수가 10 이하인 조합들을 "Rare"로 대체
#rare_labels = interaction_counts[interaction_counts <= 10].index
#train[interaction_col_name] = train[interaction_col_name].apply(lambda x: 'Rare' if x in rare_labels else x)
#test[interaction_col_name] = test[interaction_col_name].apply(lambda x: 'Rare' if x in rare_labels else x)

# 4. 상호작용항에 라벨 인코딩 적용
#interaction_col_name = ['start_end_dam', 'standby_start_fill2', 'start_end_fill2']
ordered_interaction_col_name = ['start_end_dam', 'start_end_fill2']

for i in ordered_interaction_col_name:

    interaction_le = LabelEncoder()
    train[i] = interaction_le.fit_transform(train[i])
    for label in list(test[i].unique()): #train에 없는 test값 처리
        if label not in list(interaction_le.classes_):
            interaction_le.classes_ = np.append(interaction_le.classes_,label)
    test[i] = interaction_le.transform(test[i])
    train[i] = train[i].astype("object") # 범주 변수 처리
    test[i] = test[i].astype("object") # 범주 변수 처리

모든 개별 (X,Y,Z) , X_Z_Θ 좌표 drop (상호작용 칼럼만 사용)

In [20]:
train.drop(columns=head_XYZ_col, inplace=True)
test.drop(columns=head_XYZ_col, inplace=True)

train.drop(columns=cure_XYZ_col, inplace=True)
test.drop(columns=cure_XYZ_col, inplace=True)

In [21]:
train['Mismatch_Dam_Fill1'] = (train['Production Qty Collect Result_Fill1'] != train['Production Qty Collect Result_Dam']).astype(object)
train['Mismatch_Dam_Fill2'] = (train['Production Qty Collect Result_Fill2'] != train['Production Qty Collect Result_Dam']).astype(object)
train['Mismatch_Fill1_Fill2'] = (train['Production Qty Collect Result_Fill1'] != train['Production Qty Collect Result_Fill2']).astype(object)

test['Mismatch_Dam_Fill1'] = (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Dam']).astype(object)
test['Mismatch_Dam_Fill2'] = (test['Production Qty Collect Result_Fill2'] != test['Production Qty Collect Result_Dam']).astype(object)
test['Mismatch_Fill1_Fill2'] = (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Fill2']).astype(object)

In [22]:
train['Mismatch_Receip_Dam_Fill1'] = (train['Receip No Collect Result_Fill1'] != train['Receip No Collect Result_Dam']).astype(object)
train['Mismatch_Receip_Dam_Fill2'] = (train['Receip No Collect Result_Fill2'] != train['Receip No Collect Result_Dam']).astype(object)
train['Mismatch_Receip_Fill1_Fill2'] = (train['Receip No Collect Result_Fill1'] != train['Receip No Collect Result_Fill2']).astype(object)

test['Mismatch_Receip_Dam_Fill1'] = (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Dam']).astype(object)
test['Mismatch_Receip_Dam_Fill2'] = (test['Receip No Collect Result_Fill2'] != test['Receip No Collect Result_Dam']).astype(object)
test['Mismatch_Receip_Fill1_Fill2'] = (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Fill2']).astype(object)

In [24]:
# 각 컬럼의 NA 개수 계산
na_counts = train.isna().sum()

# NA 개수가 0보다 큰 컬럼 목록 필터링
columns_with_na = na_counts[na_counts > 0].index.tolist()

# 결과 출력
print("NA 개수가 0보다 큰 컬럼들:")
columns_with_na

NA 개수가 0보다 큰 컬럼들:


[]

In [25]:
# 각 컬럼의 NA 개수 계산
na_counts = test.isna().sum()

# NA 개수가 0보다 큰 컬럼 목록 필터링
columns_with_na = na_counts[na_counts > 0].index.tolist()

# 결과 출력
print("NA 개수가 0보다 큰 컬럼들:")
columns_with_na

NA 개수가 0보다 큰 컬럼들:


['target']

### 전처리 및 파생변수 생성

In [26]:
data_type = train.dtypes.reset_index()
data_type.columns = ['col', 'type']
cat_features = list(data_type[data_type['type']=='object']['col'])
num_features = list(data_type[data_type['type']!='object']['col'])
data_type['unique'] = data_type['col'].apply(lambda x : train[x].nunique())
data_type['nullcount'] = data_type['col'].apply(lambda x : train[x].isnull().sum())
print('범주형 변수 수 : ' ,len(cat_features))
print('숫자형 변수 수 : ' ,len(num_features))

범주형 변수 수 :  59
숫자형 변수 수 :  66


In [27]:
train_num = train.loc[:,num_features]
original_cols = train_num.columns
rank = np.linalg.matrix_rank(train_num.values)
independent_cols = []
for i in range(len(original_cols)):
    sub_matrix = train_num[original_cols[:i+1]].values
    if np.linalg.matrix_rank(sub_matrix) > len(independent_cols):
        independent_cols.append(original_cols[i])
train = train.loc[:,cat_features + independent_cols]
test = test.loc[:,cat_features + independent_cols]

In [28]:
data_type = train.dtypes.reset_index()
data_type.columns = ['col', 'type']
cat_features = list(data_type[data_type['type']=='object']['col'])
num_features = list(data_type[data_type['type']!='object']['col'])
data_type['unique'] = data_type['col'].apply(lambda x : train[x].nunique())
data_type['nullcount'] = data_type['col'].apply(lambda x : train[x].isnull().sum())
print('범주형 변수 수 : ' ,len(cat_features))
print('숫자형 변수 수 : ' ,len(num_features))

범주형 변수 수 :  59
숫자형 변수 수 :  54


In [29]:
#Workmode 변수는 카테고리로 치는 것이 합당하므로 범주화
cat_add = ['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2']
for col in cat_add:
    train[col] = train[col].astype(object)
    test[col] = test[col].astype(object)
    cat_features.append(col)
    num_features.remove(col)
print('범주형 변수 수 : ' ,len(cat_features))
print('숫자형 변수 수 : ' ,len(num_features))

범주형 변수 수 :  61
숫자형 변수 수 :  52


In [30]:
#category로 취급할 것들
print(len(data_type[(data_type['unique']<=10)&(data_type['type']!='object')])) #unique값이 10개 이하이면 -> category로 취급
#cat_add = list(data_type[(data_type['unique']<=10)&(data_type['type']!='object')]['col'])

# 위에랑 사실 겹쳐서 오류나는데 오류나기전까지 들어간 얘들을 일일이 넣음
cat_add = [#'CURE END POSITION Θ Collect Result_Dam',
            'CURE SPEED Collect Result_Dam',
            'DISCHARGED SPEED OF RESIN Collect Result_Dam',
            'Stage1 Circle1 Distance Speed Collect Result_Dam',
            'Stage1 Circle2 Distance Speed Collect Result_Dam',
            'Stage1 Line1 Distance Speed Collect Result_Dam',
            'Stage1 Line2 Distance Speed Collect Result_Dam',
            'Stage1 Line4 Distance Speed Collect Result_Dam',
            'Stage2 Circle1 Distance Speed Collect Result_Dam',
            'Stage2 Circle2 Distance Speed Collect Result_Dam',
            'Stage2 Line2 Distance Speed Collect Result_Dam',
            'Stage2 Line3 Distance Speed Collect Result_Dam',
            'Stage2 Line4 Distance Speed Collect Result_Dam',
            'Stage3 Circle2 Distance Speed Collect Result_Dam',
            'Stage3 Line2 Distance Speed Collect Result_Dam',
            'Stage3 Line4 Distance Speed Collect Result_Dam',
            'THICKNESS 1 Collect Result_Dam',
            'THICKNESS 2 Collect Result_Dam',
            'THICKNESS 3 Collect Result_Dam',
            'WorkMode Collect Result_Dam',
            '1st Pressure 1st Pressure Unit Time_AutoClave',
            '2nd Pressure Unit Time_AutoClave',
            'DISCHARGED SPEED OF RESIN Collect Result_Fill1',]
            #'WorkMode Collect Result_Fill1']
for col in cat_add:
  cat_features.append(col)

25


In [31]:
for col in ['target']:
    le = LabelEncoder()
    le = le.fit(train[col])
    train[col] = le.transform(train[col])
    for label in list(test[col].unique()): #train에 없는 test값 처리
        if label not in list(le.classes_):
            le.classes_ = np.append(le.classes_,label)
    test[col] = le.transform(test[col])
cat_features.remove('target')

In [32]:
target = train['target']
train = train.drop('target', axis=1)

In [33]:
for col in cat_features:
  train[col] = train[col].astype("category")
  test[col] = test[col].astype("category")

In [34]:
train['target'] = target.apply(lambda x : 1 if x==0 else 0)

In [35]:
## 변수선택 permutation

# train.drop(columns=todrop, inplace=True)
# test.drop(columns=todrop, inplace=True)

# cat_features = [col for col in cat_features if col not in todrop]


In [36]:
tuning_train, tuning_valid = train_test_split(train, test_size=0.3, random_state=0, stratify=train['target'])
# 적합에 필요한 데이터 분리
x_train = tuning_train.drop('target', axis=1)
y_train = tuning_train['target']
# 검증에 필요한 데이터 분리
x_valid = tuning_valid.drop('target', axis=1)
y_valid = tuning_valid['target']
# set seed
SEED = 0
# class weight
class_weights = compute_class_weight(classes=[0,1], y=tuning_train['target'], class_weight='balanced')
class_weights = {i: weight for i, weight in enumerate(class_weights)}
class_weights

{0: 0.5307948631547419, 1: 8.61823708206687}

In [37]:
cat_features

['Equipment_Dam',
 'Model.Suffix_Dam',
 'Workorder_Dam',
 'Model.Suffix_AutoClave',
 'Workorder_AutoClave',
 'Chamber Temp. Judge Value_AutoClave',
 'Equipment_Fill1',
 'Model.Suffix_Fill1',
 'Workorder_Fill1',
 'Equipment_Fill2',
 'Model.Suffix_Fill2',
 'Workorder_Fill2',
 'Dam X',
 'Dam Y',
 'Dam Z',
 'Fill1 X',
 'Fill1 Y',
 'Fill1 Z',
 'Fill2 X',
 'Fill2 Y',
 'Fill2 Z',
 'Head Clean&Purge Position X Collect Result_Fill1',
 'Head Clean&Purge Position Y Collect Result_Fill1',
 'Head Clean&Purge Position Z Collect Result_Fill1',
 'Head Clean&Purge Position X Collect Result_Dam',
 'Head Clean&Purge Position Y Collect Result_Dam',
 'Head Clean&Purge Position Z Collect Result_Dam',
 'CURE  POSITION X Collect Result_Dam',
 'CURE  POSITION Θ Collect Result_Dam',
 'CURE  POSITION X Collect Result_Fill2',
 'CURE  POSITION Z Collect Result_Fill2',
 'HEAD NORMAL COORDINATE X AXIS(Stage) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Y AXIS(Stage) Collect Result_Dam',
 'HEAD NORMAL COORDINATE Z A

## 3. 모델 학습


In [38]:
space = {
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'max_depth': hp.quniform('max_depth', 5, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'n_estimators': 1000,
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

def f1_eval_metric(y_true, y_pred):
    # LightGBM은 예측 확률로 전달하므로, 이를 이진 예측으로 변환해야 합니다.
    y_pred_binary = np.round(y_pred)  # 0.5 기준으로 이진화
    f1 = f1_score(y_true, y_pred_binary)
    return 'f1_score', f1, True  

# 목적 함수 정의
def objective(params):
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])

    model = lgb.LGBMClassifier(**params, class_weight = class_weights, random_state=SEED, verbose=-1)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric=f1_eval_metric,
              categorical_feature = cat_features,
              callbacks = [lgb.early_stopping(stopping_rounds = 100), lgb.log_evaluation(period = 0)])

    preds = model.predict(x_valid)
    score = f1_score(y_valid, preds)

    # Hyperopt가 최소화를 수행하므로, 음수의 accuracy를 반환합니다.
    return {'loss': -score, 'status': STATUS_OK}



In [39]:
# Trials 객체로 결과 기록
from hyperopt.early_stop import no_progress_loss
trials = Trials()
# 최적화 수행
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=300,
            trials=trials,
            rstate=np.random.default_rng(SEED),
            early_stop_fn=no_progress_loss(50))

# 최적 파라미터 출력
print("Best hyperparameters:", best)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:                     
[74]	valid_0's binary_logloss: 0.347291	valid_0's f1_score: 0.183521
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[42]	valid_0's binary_logloss: 0.348984	valid_0's f1_score: 0.191717
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[87]	valid_0's binary_logloss: 0.314638	valid_0's f1_score: 0.207668
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[188]	valid_0's binary_logloss: 0.38227	valid_0's f1_score: 0.204931
Training until validation scores don't improve for 100 rounds                      
Early stopping, be

Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[73]	valid_0's binary_logloss: 0.367495	valid_0's f1_score: 0.19656
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[86]	valid_0's binary_logloss: 0.300533	valid_0's f1_score: 0.197688
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[118]	valid_0's binary_logloss: 0.304243	valid_0's f1_score: 0.20339
Training until validation scores don't improve for 100 rounds                      
Early stopping, best iteration is:                                                 
[119]	valid_0's binary_logloss: 0.332339	valid_0's f1_score: 0.207026
Training until validation scores don't improve for 1

In [40]:
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_weight'] = int(best['min_child_weight'])
best['n_estimators'] = 1000
best

{'colsample_bytree': 0.933211141656504,
 'learning_rate': 0.09784288222891396,
 'max_depth': 22,
 'min_child_weight': 6,
 'num_leaves': 144,
 'subsample': 0.7379599291575736,
 'n_estimators': 1000}

In [41]:
model = lgb.LGBMClassifier(**best, class_weight = class_weights, random_state=SEED, verbose=-1)
model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric=f1_eval_metric,
          categorical_feature = cat_features,
          callbacks = [lgb.early_stopping(stopping_rounds = 100), lgb.log_evaluation(period = 0)]
         )

preds = model.predict(x_valid)
score = f1_score(y_valid, preds)
print(score)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.325377	valid_0's f1_score: 0.214423
0.2144230769230769


In [42]:
x_train = train.drop('target', axis=1)
y_train = train['target']

In [43]:
# class weight
SEED = 0
n_fold = 5
class_weights = compute_class_weight(classes=[0, 1], y=train['target'], class_weight='balanced')
class_weights = {i: weight for i, weight in enumerate(class_weights)}
class_weights

{0: 0.5307946325610651, 1: 8.618297872340426}

In [44]:
from sklearn.metrics import roc_curve
model_scores = [] # model에서 뽑아내는 score
models = []
val_scores = [] # f1 score로 측정한 validation score
best_thresholds = []
pred_list = []

num = 0
cv = StratifiedKFold(n_splits = n_fold , shuffle=True, random_state = SEED)

for train_idx, valid_idx in cv.split(x_train, y_train):
    num += 1
    scores = []
    print('\n', num, '번째 fold', "="*50)
    # train valid split
    x_train_cv, x_valid_cv, y_train_cv, y_valid_cv = x_train.iloc[train_idx], x_train.iloc[valid_idx], y_train.iloc[train_idx], y_train.iloc[valid_idx]
    # model fitting
    model = lgb.LGBMClassifier(**best, class_weight = class_weights, random_state=SEED, verbose=-1)
    model.fit(x_train_cv, y_train_cv, eval_set=[(x_valid_cv, y_valid_cv)],
            eval_metric=f1_eval_metric, categorical_feature = cat_features,
            callbacks = [lgb.early_stopping(stopping_rounds = 100), lgb.log_evaluation(period = 0)]
             )
    models.append(model)

    # validation evaluate
    val_proba = model.predict_proba(x_valid_cv)[:, 1]
    # ROC Curve 계산
    fpr, tpr, thresholds = roc_curve(y_valid_cv, val_proba)
    f1_scores = []
    for thresh in thresholds:
        y_pred = [1 if y >= thresh else 0 for y in val_proba]
        f1 = f1_score(y_valid_cv, y_pred)
        f1_scores.append(f1)
    threshold = thresholds[np.argmax(f1_scores)]
    best_score = max(f1_scores)
    best_thresholds.append(threshold)
    val_scores.append(best_score) # f1 score로 측정한 validation score
    #best_thresholds.append(thresholds[index])

    # final prediction
    pred = model.predict_proba(test.drop('target',axis=1))[:, 1]
    pred_list.append(pred)


 1 번째 fold ==================================================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[151]	valid_0's binary_logloss: 0.287931	valid_0's f1_score: 0.208494

 2 번째 fold ==================================================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[203]	valid_0's binary_logloss: 0.289857	valid_0's f1_score: 0.215645

 3 번째 fold ==================================================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[187]	valid_0's binary_logloss: 0.291141	valid_0's f1_score: 0.214894

 4 번째 fold ==================================================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	valid_0's binary_logloss: 0.320262	valid_0's f1_score: 0.233533

 5 번째 fold ==================================================
Training until val

In [45]:
best_thresholds

[0.4766669920134474,
 0.5048609521529821,
 0.7308354154020255,
 0.621873939446824,
 0.6249324261928532]

In [55]:
np.mean(best_thresholds)

0.5918339450416265

In [56]:
# 이거 또는 아래 코드로 예측하는데, 제출은 아래 코드로 했음.

final_pred = np.where(sum(pred_list)/n_fold >= np.mean(best_thresholds), 1, 0)
print('cv f1 mean score :', np.mean(val_scores))
print('test에서 1로 예측된 수:', sum(final_pred))

cv f1 mean score : 0.22415407141191226
test에서 1로 예측된 수: 619


In [53]:
final_pred

array([0, 0, 0, ..., 0, 0, 0])

In [51]:
#후보군. 이거 제출 안함.

binary_predictions = []
for i in range(len(pred_list)):
    binary_pred = (pred_list[i] >= best_thresholds[i]).astype(int)
    binary_predictions.append(binary_pred)
binary_predictions = np.array(binary_predictions)
final_pred = (np.sum(binary_predictions, axis=0) >= 3).astype(int)
print('test에서 1로 예측된 수:', sum(final_pred))
print('test에서 0로 예측된 수:', len(final_pred) - sum(final_pred))

test에서 1로 예측된 수: 627
test에서 0로 예측된 수: 16734


In [48]:
# 각 pred_list 배열에 threshold 적용
binary_preds = [pred > thr for pred, thr in zip(pred_list, best_thresholds)]

# 모두 다른 항목 수 계산
np.sum(np.all(binary_preds, axis=0))

164

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
all_importances = []

# Compute permutation importance for each model on its corresponding validation set
for model, (train_idx, valid_idx) in zip(models, cv.split(x_train, y_train)):
    x_valid_cv, y_valid_cv = x_train.iloc[valid_idx], y_train.iloc[valid_idx]
    
    # Permutation importance with F1 score as the evaluation metric
    result = permutation_importance(model, x_valid_cv, y_valid_cv, scoring='f1', n_repeats=10, random_state=SEED)
    all_importances.append(result.importances_mean)  # Take the mean importance across the repeats

# Convert list to numpy array for easier averaging
all_importances = np.array(all_importances)

# Average the importances across all models
mean_importances = np.mean(all_importances, axis=0)

# Create a DataFrame for easier visualization
feature_importance_df = pd.DataFrame({
    'Feature': x_train_cv.columns,
    'Importance': mean_importances
})

# Sort by importance and select top 30 features
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True).iloc[:30]

# Visualize the top 30 feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel("Importance")
plt.title("Feature Importance (Averaged Permutation Importance)")
plt.gca().invert_yaxis()
plt.show()

In [ ]:

# Create a DataFrame for easier visualization
feature_importance_df = pd.DataFrame({
    'Feature': x_train_cv.columns,
    'Importance': mean_importances
})



# Sort by importance and select top 30 features
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True).iloc[-30:-1]

# Visualize the top 30 feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel("Importance")
plt.title("Feature Importance (Averaged Permutation Importance)")
plt.gca().invert_yaxis()
plt.show()

In [167]:
feature_importance_df = pd.DataFrame({
    'Feature': x_train_cv.columns,
    'Importance': mean_importances
})

print(feature_importance_df.to_string())

                                                       Feature  Importance
0                                                Equipment_Dam   -0.000122
1                                             Model.Suffix_Dam    0.000010
2                                                Workorder_Dam    0.098928
3                                       Model.Suffix_AutoClave    0.000251
4                                          Workorder_AutoClave    0.003505
5                          Chamber Temp. Judge Value_AutoClave    0.000725
6                                              Equipment_Fill1   -0.000233
7                                           Model.Suffix_Fill1    0.000008
8                                              Workorder_Fill1   -0.000438
9                                              Equipment_Fill2   -0.000142
10                                          Model.Suffix_Fill2    0.000000
11                                             Workorder_Fill2    0.001432
12                       

In [177]:
todrop = feature_importance_df[feature_importance_df.Importance <0].Feature.values.tolist()

## 4. 제출하기


In [57]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = final_pred
df_sub["target"] = df_sub["target"].apply(lambda x : 'AbNormal' if x==1 else 'Normal')
# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
